In [ ]:
import cobra
import numpy as np
import matplotlib as plt
import pandas as pd


# Assignment 1: Metabolic Modelling

## Task 1

- **A. In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations. [5 pt]**

    In this case, the maximal reaction activities in a linear pathway are not equal to each other. If for example we look at the lower glycolysis pathway we get the following sequence of reaction activities:
    - GAPD: 26.1
    - PGK: 23.1
    - PGM: 20.0
    - ENO: 28.8

- **B. Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions. [5 pt]**

    The first obvious reaction type are exchange reactions. If we specifically look at Ex_glc_D_e and EX_fru_e for example, they are empty and have no data. These specific reactions show the transport of that substance across the cell boundary. Gene expression data is not useful in this case for maximal activity as their rate is not only dependant on the number of expressed transport proteins. Also if there is no glucose or fructose in this case coming from an external environment the reaction rate is always zero.   

    The second reaction type would be Biomass, in this case we very clearly have BIOMASS_Ecoli_cor_w_GAM that has no data at all. The process it describes the one of cell growth, using precursor metabolites to simulate that. Since there is no gene that encodes such a process it is therefore not possible to get a maximal activity value from it only based on gene expression data. 

## Task 2

In [2]:
model = cobra.io.load_json_model('e_coli_core.json')
model

Name,e_coli_core
Memory address,768413169fd0
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"
